In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import accuracy_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! unzip /content/drive/MyDrive/copy-of-artificial-neural-networks-competition.zip

Mounted at /content/drive
Archive:  /content/drive/MyDrive/copy-of-artificial-neural-networks-competition.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train_mpst.csv          
  inflating: val.csv                 


In [ ]:
train = pd.read_csv("train_mpst.csv")
test = pd.read_csv('/content/drive/MyDrive/RA_TEMP/test.csv', error_bad_lines=False, warn_bad_lines=True)
val = pd.read_csv("val.csv")
#print(test.head())
ID = test['imdb_id']
ID =pd.DataFrame(ID)
ID.to_csv('/content/drive/MyDrive/RA_TEMP/test1.csv', index=False)

<ipython-input-22-ea389371f131>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test = pd.read_csv('/content/drive/MyDrive/RA_TEMP/test.csv', error_bad_lines=False, warn_bad_lines=True)
<ipython-input-22-ea389371f131>:2: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  test = pd.read_csv('/content/drive/MyDrive/RA_TEMP/test.csv', error_bad_lines=False, warn_bad_lines=True)


In [ ]:
X = train['plot_synopsis']
y = train.drop(['title', 'plot_synopsis','imdb_id','synopsis_source'], axis=1)
print(len(y.columns))
X_pred = test['plot_synopsis']

71


In [ ]:

mlb = MultiLabelBinarizer()
y_binary = mlb.fit_transform(y.values)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

max_len = 200
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
print(X_test_tfidf.head())
X_pred_tfidf = tfidf_vectorizer.transform(X_pred)

In [ ]:
# Neural network model
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 64
epochs = 5
history = model.fit(pad_sequences(X_train_tfidf.toarray(), maxlen=max_len),
                    y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

loss, accuracy = model.evaluate(pad_sequences(X_test_tfidf.toarray(), maxlen=max_len), y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

Epoch 1/5
95/95 [==============================] - 87s 870ms/step - loss: 0.2023 - accuracy: 0.1350 - val_loss: 0.1369 - val_accuracy: 0.1317
Epoch 2/5
95/95 [==============================] - 84s 884ms/step - loss: 0.1383 - accuracy: 0.1406 - val_loss: 0.1367 - val_accuracy: 0.1317
Epoch 3/5
95/95 [==============================] - 83s 869ms/step - loss: 0.1382 - accuracy: 0.1489 - val_loss: 0.1369 - val_accuracy: 0.1317
Epoch 4/5
95/95 [==============================] - 82s 866ms/step - loss: 0.1383 - accuracy: 0.1462 - val_loss: 0.1368 - val_accuracy: 0.1317
Epoch 5/5
60/60 [==============================] - 7s 119ms/step - loss: 0.1425 - accuracy: 0.1401
Test Loss: 0.1425, Test Accuracy: 0.1401


In [ ]:

result = []
for i in range(len(y_pred_probs)):
  prob = np.array(y_pred_probs[i])
  max_index = np.argmax(prob)
  binary_result = np.zeros_like(prob)
  binary_result[max_index] = 1
  binary_result = binary_result.tolist()
  result.append(binary_result)

type(result[0])


list

In [ ]:
Export = pd.DataFrame(result)#, axis=1)
Export.head()
Export.to_csv('/content/drive/MyDrive/ANN_by_CNN.csv', index=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Text preprocessing and vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Multi-label classification model
classifier = MultiOutputClassifier(RandomForestClassifier(random_state=42))
classifier.fit(X_train_tfidf, y_train)

# Predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluation
print(classification_report(y_test, y_pred, target_names=y.columns))

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 4.11%


In [ ]:
print(y_pred.shape)
Y = pd.DataFrame(y_pred)
Y.to_csv("/content/drive/MyDrive/AshwinBRANNFinalSubmission2.csv",index=False)